**Detector de Máscara**

Esse código é um exemplo de aplicação prática de visão computacional usando OpenCV e um modelo de deep learning treinado em Keras para detecção de máscara facial.

O código acessa a webcam, detecta rostos usando Haar Cascade, recorta e pré-processa cada face e passa pela rede neural treinada (mask_detector.h5). Em tempo real, ele exibe se a pessoa está com máscara (verde) ou sem máscara (vermelho), junto com a probabilidade da predição.

O arquivo com extensão .h5 é um formato HDF5 (Hierarchical Data Format v5), usado para armazenar modelos treinados em Keras/TensorFlow. Ele funciona como um "pacote" que guarda tanto a arquitetura da rede neural quanto os pesos aprendidos durante o treinamento.

No contexto de projetos como o de detecção de máscara facial, o .h5 é essencial porque contém:

   - Arquitetura do modelo → descreve as camadas (Dense, Convolutional, etc.), funções de ativação e conexões entre elas.

   - Pesos treinados → os parâmetros que a rede ajustou durante o treinamento, responsáveis por reconhecer padrões nas imagens.

   - Configurações adicionais → como função de perda (loss function), otimizador e métricas.

Na prática: Em vez de treinar a rede toda vez, basta carregar o arquivo .h5 já treinado.

In [ ]:
#Importações
import cv2
import numpy as np
from tensorflow.keras.models import load_model

#Carrega o modelo treinado
cnn = load_model("mask_detector.h5")

#Ajuste dos labels (este modelo pode usar 0 = Mask, 1 = No Mask)
labels_dict = {0: 'Mask', 1: 'No Mask'}
color_dict = {0: (0, 255, 0), 1: (0, 0, 255)}  # Verde = máscara, Vermelho = sem máscara

#Configuração da câmera e detector de rosto
imgsize = 4
camera = cv2.VideoCapture(0)

classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

print("▶ Webcam ligada. Pressione ESC para sair.")

#Loop de captura de vídeo
while True:
    (rval, im) = camera.read()
    if not rval:
        print("❌ Não foi possível acessar a webcam.")
        break

    #Detecção de rostos
    im = cv2.flip(im, 1, 1)
    imgs = cv2.resize(im, (im.shape[1] // imgsize, im.shape[0] // imgsize))
    face_rec = classifier.detectMultiScale(imgs)

    for i in face_rec:
        (x, y, l, w) = [v * imgsize for v in i]
        face_img = im[y:y+w, x:x+l]

        # Pré-processamento (224x224, normalizado)
        resized = cv2.resize(face_img, (224, 224))
        normalized = resized.astype("float32") / 255.0
        reshaped = np.reshape(normalized, (1, 224, 224, 3))

        # Predição
        result = cnn.predict(reshaped, verbose=0)
        label = np.argmax(result, axis=1)[0]
        prob = np.max(result) * 100  # pega a maior probabilidade

        # Desenho na imagem
        cv2.rectangle(im, (x, y), (x+l, y+w), color_dict[label], 2)
        cv2.rectangle(im, (x, y-40), (x+l, y), color_dict[label], -1)

        text = f"{labels_dict[label]} ({prob:.1f}%)"
        cv2.putText(im, text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    #Exibir janela de vídeo
    cv2.imshow('LIVE', im)

    #Encerramento
    key = cv2.waitKey(10)
    if key == 27:  # ESC
        print("👋 Encerrando...")
        break

camera.release()
cv2.destroyAllWindows()

O projeto desenvolvido demonstra de forma eficaz a aplicação prática de técnicas de visão computacional e aprendizado profundo para a detecção automática do uso de máscaras faciais em tempo real. A integração entre o modelo treinado em deep learning e a captura de vídeo via webcam permitiu uma análise instantânea e precisa, identificando se o usuário está ou não utilizando máscara.

O sistema apresentou bom desempenho mesmo em hardware limitado, graças à eficiência do modelo e à utilização otimizada dos recursos computacionais. O feedback visual, baseado em cores e probabilidades, tornou a interface intuitiva e de fácil interpretação pelo usuário.